# Assignment one - KDD cup 2014

### Read the smaller datasets into main memory, and split into training and testing

In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sqlalchemy import create_engine # database connection
from sklearn import preprocessing  
from sklearn import cross_validation

# load the data
print('loading the data...')
projects = pd.read_csv('./data/projects.csv')
outcomes = pd.read_csv('./data/outcomes.csv')
sample = pd.read_csv('./data/sampleSubmission.csv')
print('complete..')

# sort the data based on id
projects = projects.sort_values(by='projectid')
sample = sample.sort_values(by='projectid')
outcomes = outcomes.sort_values(by='projectid')

# split the training data and testing data
dates = np.array(projects.date_posted)
train_idx = np.where(dates < '2014-01-01')[0]
test_idx = np.where(dates >= '2014-01-01')[0]

# fill the missing data
projects = projects.fillna(method='pad') # fill the missing hole with the previous observation data

# set the target labels
labels = np.array(outcomes.is_exciting)

loading the data...
complete..


### Get categorical columns from projects and add them to an array projects_data

In [2]:
projects_all_columns = set(projects.columns)
projects_numeric_columns = set(projects._get_numeric_data().columns)
#projects_numeric_columns.add('date_posted')
projects_id_columns = {'projectid', 'teacher_acctid', 'schoolid', 'school_ncesid'}
projects_categorial_columns = projects_all_columns.difference(projects_id_columns).difference(projects_numeric_columns)

projects_categorial_columns = np.array(list(projects_categorial_columns))
projects_categorial_values = np.array(projects[projects_categorial_columns])

# encode the category value and reform the original data

label_encoder = LabelEncoder()
#Convert first *column* of catagorical data to numeric values
projects_data = label_encoder.fit_transform(projects_categorial_values[:,0])

#For every other *column* of categorical data, add it to the array 
for i in range(0, projects_categorial_values.shape[1]):
    projects_data = np.column_stack((projects_data, label_encoder.fit_transform(projects_categorial_values[:,i])))


###  Number of recent exciting projects a each teacher has donated to

In [3]:

disk_engine = create_engine('sqlite:///project1.db') #Create/connect to a database

teacher_donations = pd.read_sql_query("SELECT donor_acctid, COUNT(DISTINCT donations.projectid) \
                                       FROM donations \
                                       JOIN outcomes \
                                       ON donations.projectid = outcomes.projectid \
                                       WHERE is_teacher_acct='t'\
                                      AND donation_timestamp > '2013-01-01'\
                                       AND is_exciting = 't' \
                                       GROUP BY donor_acctid",disk_engine)

### Create a feature vector X

In [4]:
ID_COLUMNS = ['projectid','teacher_acctid','schoolid']   # We store the id columns so they will be easy to add and remove
X = projects[ID_COLUMNS]    # Our feature vector contains just ids

  
X = X.merge(teacher_donations,left_on='teacher_acctid',right_on='donor_acctid',how='left') # Left merge
X = X.drop('donor_acctid',axis=1) #Don't need the merged column twice
X = X.rename(columns={'projectid)': 'num_exciting_donations'})  #Rename the feature
X['num_exciting_donations'] = X['num_exciting_donations'].fillna(0)  #Any nan values mean 0 donations

### Number of exciting projects this school has had within the last year

In [5]:
merged = projects.merge(outcomes)
num_exciting_projects = merged[(merged.is_exciting=='t') & (merged.date_posted > '2013-01-01')].groupby('schoolid').count()
num_exciting_projects = num_exciting_projects.reset_index()
num_exciting_projects = num_exciting_projects[['schoolid','is_exciting']]
num_exciting_projects =  num_exciting_projects.rename(columns={'is_exciting': 'num_exciting_projects'})

### Add to feature vector X

In [6]:
X = X.merge(num_exciting_projects,left_on='schoolid',right_on='schoolid',how='left') # Left merge
X['num_exciting_projects'] = X['num_exciting_projects'].fillna(0)  #Any nan values mean 0 donations

### Number of exciting projects this teacher has had within the last year

In [7]:
num_exciting_projects_teach = merged[(merged.is_exciting=='t') & (merged.date_posted > '2013-01-01')].groupby('teacher_acctid').count()
num_exciting_projects_teach = num_exciting_projects_teach.reset_index()
num_exciting_projects_teach = num_exciting_projects_teach[['teacher_acctid','is_exciting']]
num_exciting_projects_teach =  num_exciting_projects_teach.rename(columns={'is_exciting': 'num_exciting_projects_teach'})

### Add to the feature vector X

In [8]:
X = X.merge(num_exciting_projects_teach,left_on='teacher_acctid',right_on='teacher_acctid',how='left') # Left merge
X['num_exciting_projects_teach'] = X['num_exciting_projects_teach'].fillna(0)  #Any nan values mean 0 donations

### Essay cluster success percentages

In [ ]:
clustered = pd.read_csv('.data/myclusters.csv',names = ['projectid', 'cluster'])

### Add to the feature vector X

In [ ]:
X = X.merge(clustered,left_on='projectid',right_on='projectid',how='left') # Left merge
X['cluster'] = X['cluster'].fillna(0

In [9]:
X.head()

,projectid,teacher_acctid,schoolid,num_exciting_donations,num_exciting_projects,num_exciting_projects_teach
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,61,6,5
1,00002bff514104264a6b798356fdd893,3414541eb63108700b188648f866f483,cbaae3265eda78d330cb8ab1a9217071,0,0,0
2,00002d691c05c51a5fdfbb2baef0ba25,7ad6abc974dd8b62773f79f6cbed48d5,56502bae9e97bab5eb54f9001878f469,0,0,0
3,0000b38bbc7252972f7984848cf58098,e1aa1ae5301d0cda860c4d9c89c24919,30fcfca739b17be54ce3f1ee46980340,0,0,0
4,0000ee613c92ddc5298bf63142996a5c,e0c0a0214d3c2cfdc0ab6639bc3c5342,38bb0d62aa613c2f933de56c9df855b7,0,2,1


### Length of essay feature

In [10]:
df_essay_length = pd.read_sql_query('SELECT projectid, length(essay) from essays',disk_engine)
df_essay_length = df_essay_length.sort_values(by = 'projectid')
df_essay_length = df_essay_length.fillna(0) 


### Convert the features into numpy arrays

In [11]:
num_exciting_donations = np.array(X['num_exciting_donations'])
num_exciting_projects =  np.array(X['num_exciting_projects'])
num_exciting_projects_teach =  np.array(X['num_exciting_projects_teach'])
cluster_score =  np.array(X['cluster'])
essays = np.array(df_essay_length['length(essay)'])

### Preprocess the features

In [12]:
#Preprocess the features
max_abs_scaler = preprocessing.MaxAbsScaler()  

essays = max_abs_scaler.fit_transform(essays.reshape(-1,1)) #Reshape essays and scale  
projects_data = np.column_stack((projects_data, essays))  #Add it as a column aka feature

num_exciting_donations = max_abs_scaler.fit_transform(num_exciting_donations.reshape(-1,1)) #Reshape and scale  
projects_data = np.column_stack((projects_data, num_exciting_donations))  #Add it as a column aka feature

num_exciting_projects = max_abs_scaler.fit_transform(num_exciting_projects.reshape(-1,1)) #Reshape and scale  
projects_data = np.column_stack((projects_data, num_exciting_projects))  #Add it as a column aka feature

cluster_score = max_abs_scaler.fit_transform(cluster_score.reshape(-1,1)) #Reshape and scale  
projects_data = np.column_stack((projects_data, cluster_score))  #Add it as a column aka feature

# Adding the 'num_exciting_projects_teach' results in overfitting and a poor leaderboard score, so it is commented out 


#num_exciting_projects_teach = max_abs_scaler.fit_transform(num_exciting_projects_teach.reshape(-1,1)) #Reshape and scale  
#projects_data = np.column_stack((projects_data, num_exciting_projects_teach))  #Add it as a column aka feature



### Add the numeric columns from projects 

In [13]:
projects_useful_numeric_columns = projects_numeric_columns.difference(
    {'school_zip','school_latitude','school_longitude','school_ncesid'})

for i in range(len(projects_useful_numeric_columns)):
    column = np.array(projects[list(projects_useful_numeric_columns)[i]]).reshape(-1,1)
    column = max_abs_scaler.fit_transform(column)   #Normalize
    projects_data = np.column_stack((projects_data, column))
    
projects_data = projects_data.astype(float)     #Convert everything to floats

### OHE for the categorical features

In [14]:
# One hot encoding
enc = OneHotEncoder(categorical_features=np.arange(24)) #Encode categorial features only
enc.fit(projects_data)
projects_data = enc.transform(projects_data)

In [15]:
projects_data = projects_data.tocsr() #Convert to martix that supports indexing

In [16]:
#Predicting
train = projects_data[train_idx]
test = projects_data[test_idx]

clf = LogisticRegression()

### Fit using logistic regression

In [17]:
clf.fit(train, labels=='t')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Perform 3 fold cross validation on the training set. This takes a long time to run

In [18]:
#cross_validation.cross_val_score(clf, train, labels=='t', scoring='roc_auc')

array([ 0.79283973,  0.78812177,  0.79099877])

### Make the prediction, add linear time decay, and save the predictions as a csv

In [19]:
project_date_mapping = projects[['projectid','date_posted']]
project_date_mapping.set_index('projectid')

# Calculate the decay
decay = sample.merge(project_date_mapping).drop('is_exciting',axis=1)
decay = decay.sort_values(by='date_posted')
decay['start_date'] = pd.Timestamp('2014-01-01')
decay['date_posted'] = pd.to_datetime(decay['date_posted'])
decay['factor'] = (decay.start_date-decay.date_posted).astype('timedelta64[D]')+132
decay = decay.drop(['date_posted','start_date'],axis=1)
decay = decay.set_index('projectid')

# Predict propability of exciting 
preds = clf.predict_proba(test)[:,1]

# Apply the decay and save to csv
sample = sample.set_index('projectid')
sample = sample.join(decay)
sample['is_exciting'] = preds * sample['factor']
sample = sample.drop('factor',axis=1)
sample.to_csv('predictions.csv', index = True)
sample = sample.reset_index()